# Heart Failure Prediction with Clinical Data

## Overview

Preparing the data, running basic statistics and building simple models are typical steps in a data science workflow. Here I demonstrate 3 sklearn classifiers, using clinical data as raw input to perform yes/no **Heart Failure Prediction**.

---

In [42]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [43]:
import os
import sys

DATA_PATH = "/content/drive/MyDrive/HF1/HF1-lib/"
TRAIN_DATA_PATH = DATA_PATH + "Train/"
VAL_DATA_PATH = DATA_PATH + "Val/"
    
sys.path.append("/content/drive/MyDrive/HF1/HF1-lib")

---

## 0 Raw Data

The clinical dataset is synthesized from [MIMIC-III](https://www.nature.com/articles/sdata201635).

3 CSV files are located in `TRAIN_DATA_PATH`.

In [44]:
!ls $TRAIN_DATA_PATH

event_feature_map.csv  events.csv  hf_events.csv


**events.csv**

These are patient event sequences. Each line is a tuple *(pid, event_id, vid, value)*. 

For example, 

```
33,DIAG_244,0,1
33,DIAG_414,0,1
33,DIAG_427,0,1
33,LAB_50971,0,1
33,LAB_50931,0,1
33,LAB_50812,1,1
33,DIAG_425,1,1
33,DIAG_427,1,1
33,DRUG_0,1,1
33,DRUG_3,1,1
```

- **pid**: De-identified patient identier. For example, the patient in the example above has pid 33. 
- **event_id**: Clinical event identifier. For example, DIAG_244 means the patient was diagnosed of disease with ICD9 code [244](http://www.icd9data.com/2013/Volume1/240-279/240-246/244/244.htm); LAB_50971 means that the laboratory test with code 50971 was conducted on the patient; and DRUG_0 means that a drug with code 0 was prescribed to the patient. Corresponding lab (drug) names are in `{DATA_PATH}/lab_list.txt` (`{DATA_PATH}/drug_list.txt`).
- **vid**: Ordinal visit identifier for patient visits to the healthcare provider.
- **value**: Redundant, binary, signifying associated event occurred (always 1 in the synthesized dataset).

**hf_events.csv**

Contains id code of patients who have been diagnosed with heart failure; tuples of *(pid, vid, label)*. As in, only patients with heart failure in present, and this needs to be mixed with patients not diagnosed with heart failure. For example,

```
156,0,1
181,1,1
```

The vid indicates the index of the first visit with heart failure of that patient.

**event_feature_map.csv**

The *event_feature_map.csv* is a map from an event_id to an integer index. This file contains *(idx, event_id)* pairs for all event ids.

## 1 Statistics

Each line represents an event (of which there can be multiple) that happened at an encounter.

- **Event count**: Number of events recorded for a given patient.
- **Encounter count**: Number of visits recorded for a given patient.


In [45]:
import time
import pandas as pd
import numpy as np
import datetime

def read_csv(filepath=TRAIN_DATA_PATH):
    '''
    Reads the events.csv and hf_events.csv files. 
    Variables returned from this function are passed as input to the metric functions.
    '''
    
    events = pd.read_csv(filepath + 'events.csv')
    hf = pd.read_csv(filepath + 'hf_events.csv')

    return events, hf

def event_count_metrics(events, hf):
    '''
    Returns event count metrics.
    Event count is defined as the number of events recorded for a given patient.
    '''

    avg_hf_event_count = None
    max_hf_event_count = None
    min_hf_event_count = None
    avg_norm_event_count = None
    max_norm_event_count = None
    min_norm_event_count = None
    
    df1 = hf_events_table(events, hf)
    avg_hf_event_count = df1['eventcount_fromevents'].mean()
    max_hf_event_count = df1['eventcount_fromevents'].max()
    min_hf_event_count = df1['eventcount_fromevents'].min()
    
    df2 = norm_events_table(events, hf)
    avg_norm_event_count = df2['eventcount_fromevents'].mean()
    max_norm_event_count = df2['eventcount_fromevents'].max()
    min_norm_event_count = df2['eventcount_fromevents'].min()

    return avg_hf_event_count, max_hf_event_count, min_hf_event_count, \
           avg_norm_event_count, max_norm_event_count, min_norm_event_count

def encounter_count_metrics(events, hf):
    '''
    Returns encounter count metrics.
    Encounter count is defined as the number of visits recorded for a given patient. 
    '''

    avg_hf_encounter_count = None
    max_hf_encounter_count = None
    min_hf_encounter_count = None
    avg_norm_encounter_count = None
    max_norm_encounter_count = None
    min_norm_encounter_count = None
    
    df1 = hf_visit_table(events, hf) 
    avg_hf_encounter_count = df1['visitcount_fromevents'].mean()
    max_hf_encounter_count = df1['visitcount_fromevents'].max()
    min_hf_encounter_count = df1['visitcount_fromevents'].min()
    
    df2 = norm_visit_table(events, hf) 
    avg_norm_encounter_count = df2['visitcount_fromevents'].mean()
    max_norm_encounter_count = df2['visitcount_fromevents'].max()
    min_norm_encounter_count = df2['visitcount_fromevents'].min()

    return avg_hf_encounter_count, max_hf_encounter_count, min_hf_encounter_count, \
           avg_norm_encounter_count, max_norm_encounter_count, min_norm_encounter_count


def left_anti(events, hf):
    '''
    This gives full data on the 1040 non-hf patients
    '''
    df = pd.merge(left=events,
                  right=hf, 
                  on ='pid', 
                  how ='left',
                  indicator=True,
                  suffixes=('_fromevents', '_fromhf')) \
                  .query('_merge == "left_only"') \
                  .drop('_merge', 1)
    return df

def left_join_pid(events, hf):
    '''
    This gives full data on the 4000 patients, with the hf diagnosis visit id on the 2060 patients
    '''
    df = pd.merge(left=events, 
                right=hf, 
                on ='pid', 
                how ='left',
                suffixes=('_fromevents', '_fromhf'))
    return df

def right_join_pid(events, hf):
    '''
    This gives full data on the 2060 hf patients
    '''
    df = pd.merge(left=events, 
                right=hf, 
                on ='pid', 
                how ='right',
                suffixes=('_fromevents', '_fromhf'))
    return df

def hf_events_table(events, hf):
    '''
    Returns (non-unique) event counts in 'event_id' column
    '''
    df = right_join_pid(events, hf)
    groupby1 = df.groupby(["pid"], as_index=False).count() \
      .rename(columns={'event_id':'eventcount_fromevents'})
    return groupby1

def norm_events_table(events, hf):
    '''
    Returns (non-unique) event counts in 'event_id' column
    '''
    df = left_anti(events, hf)
    groupby3 = df.groupby(["pid"], as_index=False).count() \
      .rename(columns={'event_id':'eventcount_fromevents'})
    return groupby3

def hf_visit_table(events, hf):
    '''
    Returns unique visit counts in 'vid_fromevents' column, NOT vids
    THIS MAY BE A PROBLEM IF THE VIDS SKIP NUMBERS, I'M CORRECTING THIS HERE
    '''
    df = right_join_pid(events, hf)
    # group on unique visits and aggregate-count
    groupby1 = df.groupby(["pid", "vid_fromevents"], as_index=False).count() \
      .rename(columns={'vid_fromevents':'visitcount_fromevents'}) \
      .drop(columns=['value', 'event_id', 'vid_fromhf', 'label'])
    # group on patient for unique visit counts
    groupby2 = groupby1.groupby(["pid"]).count()
    return groupby2

def norm_visit_table(events, hf):
    '''
    Returns unique visit counts in 'vid_fromevents' column, NOT vids
    '''
    df = left_anti(events, hf)
    groupby3 = df.groupby(["pid", "vid_fromevents"], as_index=False).count() \
      .rename(columns={'vid_fromevents':'visitcount_fromevents'}) \
      .drop(columns=['value', 'event_id', 'vid_fromhf', 'label'])
    groupby4 = groupby3.groupby(["pid"]).count()
    return groupby4

In [46]:
### FINDINGS
## There are 4000 patients in total
## There are 2960 patients with hf
## 1040 patients without hf
## There is 100% overlap of hf on events
# events, hf = read_csv(TRAIN_DATA_PATH)
# norm_visit_table(events, hf)

In [47]:
if __name__=='__main__':

  events, hf = read_csv(TRAIN_DATA_PATH)

  #Compute the event count metrics
  start_time = time.time()
  event_count = event_count_metrics(events, hf)
  end_time = time.time()
  print(("Time to compute event count metrics: " + str(end_time - start_time) + "s"))
  print(event_count)

  #Compute the encounter count metrics
  start_time = time.time()
  encounter_count = encounter_count_metrics(events, hf)
  end_time = time.time()
  print(("Time to compute encounter count metrics: " + str(end_time - start_time) + "s"))
  print(encounter_count)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:82: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


Time to compute event count metrics: 0.4409446716308594s
(188.9375, 2046, 28, 118.64423076923077, 1014, 6)
Time to compute encounter count metrics: 0.44466209411621094s
(2.8060810810810812, 34, 2, 2.189423076923077, 11, 1)


## 2 Feature construction

Here I convert raw data into a SVM Light format for sparse dimension data.

I create some important objects:

- **Index vid**: Index vid:
  - For heart failure patients: Index vid is the vid of the first visit with heart failure for that patient (i.e., vid field in *hf_events.csv*). 
  - For normal patients: Index vid is merely the vid of the last visit for that patient (i.e., vid field in *events.csv*). 
- **Observation Window**: The time interval of modeling interest per patient.
  - For heart failures: the visits prior to the visit with HF diagnosis (we don't care about what happens after HF is already known)
  - For normals: 
- **Prediction Window**: The point where a HF prediction will be made.

If index vid is 3, visits with vid 0, 1, 2 are within the observation window, and the prediction window is between visit 2 and 3.

### 2.1 Index vid


In [48]:
import pandas as pd
import datetime


def read_csv(filepath=TRAIN_DATA_PATH):
    events = pd.read_csv(filepath + 'events.csv')
    hf = pd.read_csv(filepath + 'hf_events.csv')
    feature_map = pd.read_csv(filepath + 'event_feature_map.csv')

    return events, hf, feature_map


def calculate_index_vid(events, hf):
    '''
    Returns pandas dataframe with header `['pid', 'indx_vid']`.

    Suggested steps:
        1. Create list of normal patients (hf_events.csv only contains information about heart failure patients).
        2. Split events into two groups based on whether the patient has heart failure or not.
        3. Calculate index vid for each patient.
    '''

    indx_vid = ''
    
    # 2960 hfs: get last visit from hf table as indx_vid (max is unnecessary)
    df_hf_vid_max = right_join_pid(events, hf).groupby(['pid'], as_index=True).max()
    # set the index visit as the vid from hf
    df_hf_vid_max['indx_vid'] = df_hf_vid_max['vid_fromhf']

    # 1040 normies: get last visit from norm table as indx_vid
    '''
    In reviewing my code, I identify this point as important choice
    MAJOR SOURCE OF CONFUSION: CHOICE BETW (TOTAL VISIT COUNT - 1) OR ORIG VID
    --------------------------------------------------------------------------
    '''
    choice = 'orig vid'
    if choice == 'orig vid':
      # Orig vid approach
      df_norm_vid_max = left_anti(events, hf).groupby(['pid'], as_index=True).max()
      df_norm_vid_max['indx_vid'] = df_norm_vid_max['vid_fromevents']
    elif choice == 'visitcount':
      # Total visit count approach
      df_norm_vid_max = norm_visit_table(events, hf).groupby(['pid'], as_index=True).max()
      df_norm_vid_max['indx_vid'] = df_norm_vid_max['visitcount_fromevents'] - 1
    else:
      raise ValueError('choice variable set incorrectly')

    indx_vid = df_hf_vid_max.append(df_norm_vid_max).reset_index().drop(columns=['event_id','vid_fromevents','value','vid_fromhf','label'])

    return indx_vid

In [49]:
### TEST CELL
## There are 4000 patients in total
## There are 2960 patients with hf
## 1040 patients without hf
## There is 100% overlap of hf on events

events, hf, _ = read_csv(TRAIN_DATA_PATH)
df = calculate_index_vid(events, hf)
df[df.pid == 78]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:82: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


,pid,indx_vid
2960,78,1


### 2.2 Filter events

This removes events outside the observation window.

In [50]:
def filter_events(events, indx_vid):
    '''
    Returns dataframe of the observation window data for HF patients
    `['pid', 'event_id', 'value']` up to but not including the index visit.

      1. Join indx_vid with events on pid
      2. Filter events
    
    indx_vid columns = index, pid, indx_vid
    events columns = pid, event_id, vid, value
    '''

    filtered_events = None
    
    '''
    ---------------------------
    Key choice at this point:
    FILTER (inner join) BY INDX_VID
    ---------------------------
    '''
    # join dfs on two columns
    inner_join = pd.merge(left=indx_vid, 
            right=events, 
            on=['pid'], 
            how ='inner')
    
    # filter all event_ids less than idx_vid
    filtered_events = inner_join[(inner_join.vid < inner_join.indx_vid)]

    return filtered_events.drop(columns=['indx_vid', 'vid'])

In [51]:
'''
### TEST
events, hf, feature_map = read_csv(TRAIN_DATA_PATH)
index = calculate_index_vid(events, hf)
# left_outer = pd.merge(left=indx_vid, 
#         right=events, 
#         left_on=['pid'], 
#         right_on=['pid'],
#         how ='left',
#         suffixes=('_fromidx', '_fromevents'))
# df = left_outer[(left_outer.vid < left_outer.indx_vid)]
# df[df.pid == 78]
filter_events(events, index)
'''

"\n### TEST\nevents, hf, feature_map = read_csv(TRAIN_DATA_PATH)\nindex = calculate_index_vid(events, hf)\n# left_outer = pd.merge(left=indx_vid, \n#         right=events, \n#         left_on=['pid'], \n#         right_on=['pid'],\n#         how ='left',\n#         suffixes=('_fromidx', '_fromevents'))\n# df = left_outer[(left_outer.vid < left_outer.indx_vid)]\n# df[df.pid == 78]\nfilter_events(events, index)\n"

### 2.3 Aggregate events

I will use count of events as a feature, and so prepare it here:

So for this patient...

```
33,DIAG_244,0,1
33,LAB_50971,0,1
33,LAB_50931,0,1
33,LAB_50931,0,1
33,DIAG_244,1,1
33,DIAG_427,1,1
33,DRUG_0,1,1
33,DRUG_3,1,1
33,DRUG_3,1,1
```
...will become *(event_id, value)* 

```
(DIAG_244, 2.0)
(LAB_50971, 1.0)
(LAB_50931, 2.0)
(DIAG_427, 1.0)
(DRUG_0, 1.0)
(DRUG_3, 2.0)
```

Then index the event codes using *event_feature_map.csv*.

```
(146, 2.0)
(1434, 1.0)
(1429, 2.0)
(304, 1.0)
(898, 1.0)
(1119, 2.0)
```

Also, I min-max normalize to work in 0-1 space. For simplicity I define min(x) as zero, so just $x$/$max(x)$).

In [52]:
def aggregate_events(filtered_events_df, _, feature_map_df):
    '''
    Returns dataframe with header `['pid', 'feature_id', 'feature_value']`

      1. Lookup event_id's from event_feature_map.csv.
      2. Aggregate events with count
      3. Normalize with min-max (the min value is assumed zero).

    filtered_events_df > 'pid', 'event_id', 'value'
    feature_map_df > 'idx' , 'event_id'
    '''
    
    aggregated_events = None
  
    ### Step 0: remap eventids to integers
    feature_map_df = feature_map_df.rename(columns={'idx':'feature_id'})

    inner_join = pd.merge(left=filtered_events_df, 
        right=feature_map_df, 
        on=['event_id'], 
        how ='inner').drop(columns=['value'])

    ### Step 1: get the per-patient frequency of (patient-)events
    inner_join['freq'] = 1
    '''
    NOTE: Here I decide to keep duplicate events per visit. But I would probably 
    remove duplicates if using this model further.
    --------------------------------------------------------------------------
    '''
    counts = inner_join.groupby(['pid', 'feature_id'], as_index=True).count() \
      .reset_index() 

    ### Step 2: add min and max columns
    # 1252 rows or events present in filtered events
    df_maxs = counts.groupby(['feature_id'], as_index=True).max() \
      .rename(columns={'freq':'freq_max', 'pid':'pid_max'})

    # leftouter join to attach max values
    # 154754 rows preserved
    inner_join2 = pd.merge(left=counts,
                          right=df_maxs,
                          on=['feature_id'],
                          how = 'inner',
                          suffixes=('_orig', '_max')).rename(columns={'freq':'freq_orig'})

    ### Step 3: add new column- normalize against max AND min values
    freq_min = 0
    inner_join2['feature_value'] = (inner_join2['freq_orig'] - freq_min) / (inner_join2['freq_max'] - freq_min)

    inner_join2 = inner_join2.fillna(0)

    #cleanup
    aggregated_events = inner_join2 \
      .drop(columns=['freq_max','freq_orig', 'pid_max','event_id_orig', 'event_id_max'])

    return aggregated_events

### 2.4 Convert to SVMLight format

If we have a lot of features but not many hits (i.e. it has only a few nonzero elements), its best to compress the space.

```
<line> .=. <target> <feature>:<value> <feature>:<value>
<target> .=. 1 | 0
<feature> .=. <integer>
<value> .=. <float>
```

Feature/value pairs MUST be ranked by ascending feature number. The SVMLight format looks like: 

```
1 2:0.5 3:0.12 10:0.9 2000:0.3
0 4:1.0 78:0.6 1009:0.2
1 33:0.1 34:0.98 1000:0.8 3300:0.2
1 34:0.1 389:0.32
```

where each row is a patient, label 1 indicates heart failure and feature-value pairs follow **sorted** by the feature index (idx) value.

In [58]:
from sklearn.datasets import load_svmlight_file

def bag_to_svmlight(input):
    return ' '.join(( "%d:%f" % (fid, float(fvalue)) for fid, fvalue in input))

#input: features and label stored in the svmlight_file
#output: X_train, Y_train
#Note: hardcode the number of features
def get_data_from_svmlight(svmlight_file):
    data_train = load_svmlight_file(svmlight_file,n_features=1473)
    X_train = data_train[0]
    Y_train = data_train[1]
    return X_train, Y_train

In [61]:
import collections

def create_features(events_in, hf_in, feature_map_in):
    '''
    Returns two dicts:
    1. patient_features: Key is pid and value is array of tuples(feature_id, feature_value). 
    2. hf: Key is pid and value is heart failure label.

    indx_vid > index, pid, indx_vid
    events_in > pid, event_id, vid, value
    hf_in > 'pid', 'vid', 'label'
    feature_map_in > 'idx' , 'event_id'
    filtered_events > 'pid', 'event_id', 'value'
    aggregated_events > ['pid', 'feature_id', 'feature_value']
    '''

    indx_vid = calculate_index_vid(events_in, hf_in)

    #Filter events in the observation window
    filtered_events = filter_events(events_in, indx_vid)

    #Aggregate the event values for each patient 
    aggregated_events = aggregate_events(filtered_events, _, feature_map_in)

    patient_features = None
    hf = None
  
    # turn the dataframes into dicts
    aggregated_events = pd.DataFrame(aggregated_events.set_index('pid').apply(tuple, axis=1)).sort_values(by=0,ascending=True)
    patient_features = aggregated_events.stack().groupby(level=0).apply(list).to_dict()

    hf = hf_in
    hf = hf.drop(columns=['vid']).groupby('pid').count()
    hf = hf.sort_index().stack().groupby(level=0).apply(int).to_dict()
  
    return patient_features, hf

def save_svmlight(patient_features, hf, op_file):
    '''
    Creates op_file in svmlight format
    '''
    
    deliverable = open(op_file, 'wb')

    # Create list of strings
    l = [( str(hf[pid] if pid in hf else '0') + ' ' + bag_to_svmlight(tuples)) for pid, tuples in patient_features.items()]
    # write to file
    for example in l:
      label = example.split(" ", 1)[0]
      features = example.split(" ", 1)[1]
      deliverable.write(bytes(f"{label} {features} \n", 'utf-8'))
    return None

In [62]:
if __name__ == '__main__':
    events_in, hf_in, feature_map_in = read_csv(TRAIN_DATA_PATH)
    patient_features, hf = create_features(events_in, hf_in, feature_map_in)
    save_svmlight(patient_features, hf, 'features_svmlight.train')
    
    events_in, hf_in, feature_map_in = read_csv(VAL_DATA_PATH)
    patient_features, hf = create_features(events_in, hf_in, feature_map_in)
    save_svmlight(patient_features, hf, 'features_svmlight.val')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:82: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


## 3 Predictive Modeling


### 3.1 Model Build

Here I implement Logistic Regression, SVM and a Decision Tree with sklearn and evaluate on the test dataset.

In [64]:
import numpy as np
from sklearn.datasets import load_svmlight_file
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import *


RANDOM_STATE = 545510477


def logistic_regression_pred(X_train, Y_train):
    """
    Returns predictions with a binary logistic classifier
    """

    clfr = LogisticRegression(random_state=RANDOM_STATE)
    clfr.fit(X=X_train, y=Y_train)
    Y_pred = clfr.predict(X_train)
    return Y_pred

  
def svm_pred(X_train, Y_train):
    """
    Returns predictions with an SVM classifier
    """

    clfr = LinearSVC(random_state=RANDOM_STATE)
    clfr.fit(X=X_train, y=Y_train)
    Y_pred = clfr.predict(X_train)
    return Y_pred

    
def decisionTree_pred(X_train, Y_train):
    """
    Returns predictions with a decision tree of max_depth 5
    """

    clfr = DecisionTreeClassifier(random_state=RANDOM_STATE, max_depth=5)
    clfr.fit(X=X_train, y=Y_train)
    Y_pred = clfr.predict(X_train)
    return Y_pred

    

def classification_metrics(Y_pred, Y_true):
    """
    input: Y_pred, Y_true
    output: accuracy, precision, recall, f1-score
    """
    
    # %age of correct Y_pred (1 or 0) over numrows
    acc = accuracy_score(y_true=Y_true, y_pred=Y_pred)
    # count of correct (Y_pred == 1) over len(Y_pred)
    precision = precision_score(y_true=Y_true, y_pred=Y_pred)
    # count of correct (Y_pred == 1) over that plus incorrect (Y_pred == 0)
    recall = recall_score(y_true=Y_true, y_pred=Y_pred)
    # harmonic mean of precision and recall n / (1/x) * (1/y)
    f1score = f1_score(y_true=Y_true, y_pred=Y_pred)

    return acc, precision, recall, f1score

    
def display_metrics(classifierName, Y_pred, Y_true):
    print("______________________________________________")
    print(("Classifier: "+classifierName))
    acc, precision, recall, f1score = classification_metrics(Y_pred,Y_true)
    print(("Accuracy: "+str(acc)))
    print(("Precision: "+str(precision)))
    print(("Recall: "+str(recall)))
    print(("F1-score: "+str(f1score)))
    print("______________________________________________")
    print("")

    
if __name__ == '__main__': 
    X_train, Y_train = get_data_from_svmlight("features_svmlight.train")

    display_metrics("Logistic Regression", logistic_regression_pred(X_train, Y_train), Y_train)
    display_metrics("SVM",svm_pred(X_train, Y_train),Y_train)
    display_metrics("Decision Tree", decisionTree_pred(X_train, Y_train), Y_train)

______________________________________________
Classifier: Logistic Regression
Accuracy: 0.856338028169014
Precision: 0.8357933579335793
Recall: 0.937888198757764
F1-score: 0.8839024390243903
______________________________________________

______________________________________________
Classifier: SVM
Accuracy: 0.9070422535211268
Precision: 0.896484375
Recall: 0.9503105590062112
F1-score: 0.9226130653266331
______________________________________________

______________________________________________
Classifier: Decision Tree
Accuracy: 0.703420523138833
Precision: 0.6657355679702048
Recall: 0.9868875086266391
F1-score: 0.7951070336391437
______________________________________________



In [66]:
import numpy as np
from sklearn.datasets import load_svmlight_file
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import *


RANDOM_STATE = 545510477


def logistic_regression_pred(X_train, Y_train, X_test):
    """
    Returns predictions from a logistic regression classifier on the test set 
    
    input: X_train, Y_train and X_test
    output: Y_pred
    """
    
    clfr = LogisticRegression(random_state=RANDOM_STATE)
    clfr.fit(X=X_train, y=Y_train)
    Y_pred = clfr.predict(X_test)
    return Y_pred
    


def svm_pred(X_train, Y_train, X_test):    
    """
    Returns predictions from an SVM classifier on the test set 
    
    input: X_train, Y_train and X_test
    output: Y_pred
    """
    
    clfr = LinearSVC(random_state=RANDOM_STATE)
    clfr.fit(X=X_train, y=Y_train)
    Y_pred = clfr.predict(X_test)
    return Y_pred

    
#input: X_train, Y_train and X_test
#output: Y_pred
def decisionTree_pred(X_train, Y_train, X_test):
    """
    Returns predictions from a decision tree classifier on the test set 
    
    input: X_train, Y_train and X_test
    output: Y_pred
    """
    
    clfr = DecisionTreeClassifier(random_state=RANDOM_STATE, max_depth=5)
    clfr.fit(X=X_train, y=Y_train)
    Y_pred = clfr.predict(X_test)
    return Y_pred




def classification_metrics(Y_pred, Y_true):  
    """
    Returns metrics.

    #input: Y_pred,Y_true
    #output: accuracy, precision, recall, f1-score
    """
    
    # %age of correct Y_pred (1 or 0) over numrows
    acc = accuracy_score(y_true=Y_true, y_pred=Y_pred)
    # count of correct (Y_pred == 1) over len(Y_pred)
    precision = precision_score(y_true=Y_true, y_pred=Y_pred)
    # count of correct (Y_pred == 1) over that plus incorrect (Y_pred == 0)
    recall = recall_score(y_true=Y_true, y_pred=Y_pred)
    # harmonic mean of precision and recall n / (1/x) * (1/y)
    f1score = f1_score(y_true=Y_true, y_pred=Y_pred)

    return acc, precision, recall, f1score
    
    
def display_metrics(classifierName, Y_pred, Y_true):
    print("______________________________________________")
    print(("Classifier: "+classifierName))
    acc, precision, recall, f1score = classification_metrics(Y_pred,Y_true)
    print(("Accuracy: "+str(acc)))
    print(("Precision: "+str(precision)))
    print(("Recall: "+str(recall)))
    print(("F1-score: "+str(f1score)))
    print("______________________________________________")
    print("")

    
if __name__ == '__main__': 
    X_train, Y_train = get_data_from_svmlight("features_svmlight.train")
    X_test, Y_test = get_data_from_svmlight(os.path.join("features_svmlight.val"))

    display_metrics("Logistic Regression", logistic_regression_pred(X_train, Y_train, X_test), Y_test)
    display_metrics("SVM", svm_pred(X_train, Y_train, X_test), Y_test)
    display_metrics("Decision Tree", decisionTree_pred(X_train, Y_train, X_test), Y_test)

______________________________________________
Classifier: Logistic Regression
Accuracy: 0.6937086092715232
Precision: 0.7345360824742269
Recall: 0.776566757493188
F1-score: 0.7549668874172186
______________________________________________

______________________________________________
Classifier: SVM
Accuracy: 0.640728476821192
Precision: 0.7038043478260869
Recall: 0.7057220708446866
F1-score: 0.7047619047619047
______________________________________________

______________________________________________
Classifier: Decision Tree
Accuracy: 0.6821192052980133
Precision: 0.6611418047882136
Recall: 0.9782016348773842
F1-score: 0.789010989010989
______________________________________________



### 3.2 Model Validation

Here I implement cross-validation:

- K-fold: Divide all the data into $k$ groups of samples. Each time $\frac{1}{k}$ samples will be used as test data and the remaining samples as training data.
- Randomized K-fold: Iteratively random shuffle the whole dataset and use top specific percentage of data (I use 20%) as training and the rest as test. 

In [68]:
from sklearn.model_selection import KFold, ShuffleSplit
from numpy import mean


RANDOM_STATE = 545510477


def get_f1_kfold(X, Y, k=5):
    """
    Returns mean f1 score of all the fold runs. Repeats the train/test process k times.

    Input: test and train row indices for each run
    """
    
    acc = []
    f1score = []
    # create the index mask
    for i, (train_index, test_index) in enumerate(KFold(n_splits=k).split(X, Y)):
      X_train, Y_train = X[train_index], Y[train_index]
      X_test, Y_test = X[test_index], Y[test_index]
      clfr = LinearSVC(random_state=RANDOM_STATE)
      clfr.fit(X=X_train, y=Y_train)
      Y_pred = clfr.predict(X_test)
      acc.append(accuracy_score(y_true=Y_test, y_pred=Y_pred))
      f1score.append(f1_score(y_true=Y_test, y_pred=Y_pred))
      # print(f'iteration: {i} accuracy {acc[i]} f1score {f1score[i]}')

    return np.mean(np.array(f1score)) #, np.mean(np.array(acc))

    
def get_f1_randomisedCV(X, Y, iterNo=5, test_percent=0.20):
    """
    Returns mean f1 score of all the fold runs. Repeats the train/test process iterNo times.

    Input: test and train row indices for each run
    """

    acc = []
    f1score = []
    # create the index mask
    for i, (train_index, test_index) in enumerate(ShuffleSplit(n_splits=iterNo, test_size=test_percent, random_state=RANDOM_STATE).split(X, Y)):
      X_train, Y_train = X[train_index], Y[train_index]
      X_test, Y_test = X[test_index], Y[test_index]
      clfr = LinearSVC(random_state=RANDOM_STATE)
      clfr.fit(X=X_train, y=Y_train)
      Y_pred = clfr.predict(X_test)
      acc.append(accuracy_score(y_true=Y_test, y_pred=Y_pred))
      f1score.append(f1_score(y_true=Y_test, y_pred=Y_pred))
      # print(f'iteration: {i} accuracy {acc[i]} f1score {f1score[i]}')
      
    return np.mean(np.array(f1score)) #, np.mean(np.array(acc))

    
if __name__ == '__main__': 
    X,Y = get_data_from_svmlight("features_svmlight.train")
    print("Classifier: SVD")
    f1_k = get_f1_kfold(X,Y)
    print(("Average F1 Score in KFold CV: "+str(f1_k)))
    f1_r = get_f1_randomisedCV(X,Y)
    print(("Average F1 Score in Randomised CV: "+str(f1_r)))

Classifier: SVD
Average F1 Score in KFold CV: 0.7258461959533061
Average F1 Score in Randomised CV: 0.7195678940019832
